# **Performing one-hot encoding of frequent categories**

One-hot encoding represents each category of a categorical variable with a binary variable.
Hence, one-hot encoding of highly cardinal variables or datasets with multiple categorical
features can expand the feature space dramatically. To reduce the number of binary
variables, we can perform one-hot encoding of the most frequent categories only. In this work, we will learn how to implement one-hot encoding of the most popular
categories using pandas and Feature-engine.

In [1]:
pip install feature-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 5.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

# to split the datasets
from sklearn.model_selection import train_test_split

# for one hot encoding with feature-engine
from feature_engine.encoding import OneHotEncoder

In [3]:
# let's load the data set

data = pd.read_csv('creditApprovalUCI.csv')

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [4]:
# make a list with the categorical variables

vars_categorical = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [5]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['A16'], axis=1),  # predictors
    data['A16'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((483, 15), (207, 15))

# **One Hot Encoding of top categories with pandas**

In [6]:
# let's inspect the unique categories of A6

X_train['A6'].unique()

array(['c', 'q', 'w', 'ff', 'm', 'i', 'e', 'cc', 'x', 'd', 'k', 'j',
       'Missing', 'aa', 'r'], dtype=object)

In [7]:
# let's find the top 5 most frequent categories in A6

X_train['A6'].value_counts().sort_values(ascending=False).head(5)

c     93
q     56
w     48
i     41
ff    38
Name: A6, dtype: int64

In [8]:
# let's make a list with the most frequent categories in A6

top_5 = [
    x for x in X_train['A6'].value_counts().sort_values(
        ascending=False).head(5).index
]

top_5

['c', 'q', 'w', 'i', 'ff']

In [9]:
# and now let's create 5 binary variables to encode A6
# in train and test sets

for label in top_5:
    X_train['A6' + '_' + label] = np.where(
        X_train['A6'] == label, 1, 0)

    X_test['A6' + '_' + label] = np.where(
        X_test['A6'] == label, 1, 0)

In [10]:
# let's visualise the result

print(X_train[['A6'] + ['A6'+'_'+c for c in top_5]].head(10))

     A6  A6_c  A6_q  A6_w  A6_i  A6_ff
596   c     1     0     0     0      0
303   q     0     1     0     0      0
204   w     0     0     1     0      0
351  ff     0     0     0     0      1
118   m     0     0     0     0      0
247   q     0     1     0     0      0
652   i     0     0     0     1      0
513   e     0     0     0     0      0
230  cc     0     0     0     0      0
250   e     0     0     0     0      0


# **One Hot Encoding of top categories with Feature-Engine**

In [11]:
# let's divide in train and test sets (again)

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['A16'], axis=1),  # predictors
    data['A16'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

In [13]:
ohe_enc = OneHotEncoder(

    # to indicate how many top categories
    top_categories=5,

    # we can select which variables to encode
    variables=['A6', 'A7'],

    # to indicate if we drop one of the binaries
    drop_last=False)

ohe_enc.fit(X_train)

OneHotEncoder(top_categories=5, variables=['A6', 'A7'])

In [14]:
# the encoder stores the variables it will encode

ohe_enc.variables

['A6', 'A7']

In [15]:
# the encoder stores the most frequent labels per variable

ohe_enc.encoder_dict_

{'A6': ['c', 'q', 'w', 'i', 'ff'], 'A7': ['v', 'h', 'ff', 'bb', 'z']}

In [16]:
# let's transform train and test sets

X_train_enc = ohe_enc.transform(X_train)
X_test_enc = ohe_enc.transform(X_test)

In [17]:
# let's inspect the result

X_train_enc.head()

,A1,A2,A3,A4,A5,A8,A9,A10,A11,A12,...,A6_c,A6_q,A6_w,A6_i,A6_ff,A7_v,A7_h,A7_ff,A7_bb,A7_z
596,a,46.08,3.000,u,g,2.375,t,t,8,t,...,1,0,0,0,0,1,0,0,0,0
303,a,15.92,2.875,u,g,0.085,f,f,0,f,...,0,1,0,0,0,1,0,0,0,0
204,b,36.33,2.125,y,p,0.085,t,t,1,f,...,0,0,1,0,0,1,0,0,0,0
351,b,22.17,0.585,y,p,0.000,f,f,0,f,...,0,0,0,0,1,0,0,1,0,0
118,b,57.83,7.040,u,g,14.000,t,t,6,t,...,0,0,0,0,0,1,0,0,0,0


In [18]:
X_test_enc.head()

,A1,A2,A3,A4,A5,A8,A9,A10,A11,A12,...,A6_c,A6_q,A6_w,A6_i,A6_ff,A7_v,A7_h,A7_ff,A7_bb,A7_z
14,a,45.83,10.50,u,g,5.000,t,t,7,t,...,0,1,0,0,0,1,0,0,0,0
586,b,64.08,20.00,u,g,17.500,t,t,9,t,...,0,0,0,0,0,0,1,0,0,0
140,a,31.25,3.75,u,g,0.625,t,t,9,t,...,0,0,0,0,0,0,1,0,0,0
492,b,39.25,9.50,u,g,6.500,t,t,14,f,...,0,0,0,0,0,1,0,0,0,0
350,a,26.17,2.00,u,g,0.000,f,f,0,t,...,0,0,0,0,0,0,0,0,0,0
